In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gensim
import gensim.downloader
import scipy 

In [2]:
vectors = gensim.downloader.load('glove-wiki-gigaword-300')

In [13]:
df_raw = pd.read_csv("https://raw.githubusercontent.com/SeanMKoh/Student-Accommodation-Review-Research/main/AccomReviewSample.csv", index_col=0)
df_raw.drop_duplicates(subset=['Review'], inplace=True)
df_raw.head()

,Accom,Review
0,iQ Weaver Place,IQ weaver place has been an amazing place to r...
1,Julian Court,I am living in classic studio with a window fa...
2,Julian Court,I have a very nice experience staying in Julia...
3,Julian Court,"As a self-contained studio apartment, Julian C..."
4,Julian Court,"I like the young lady in reception, she is alw..."


In [14]:
df_pro = pd.read_csv("https://raw.githubusercontent.com/SeanMKoh/Student-Accommodation-Review-Research/main/WordTagging.csv", index_col=0)
df_pro.drop_duplicates(subset=['Review'], inplace=True)
df_pro.head()

,Review,Nouns
0,iq weaver place has been an amazing place to r...,"['iq', 'weaver', 'place', 'place', 'while']"
1,moved here in september and spent most of lock...,"['lockdown', 'space']"
2,amazing on site staff who are always ready and...,"['site', 'staff', 'type', 'assistance']"
3,would definitely recommend staying here for un...,"['students', 'book', 'accommodation']"
4,i am living in classic studio with a window fa...,"['i', 'studio', 'window', 'facing']"


In [231]:
cat_list = ['space', 
            'personnel',
            'social',
            'cleanliness',
            'distance'
           ]
cat_vector_list = []
for i in cat_list:
    cat_vector_list.append(vectors[i])

In [4]:
def vector_difference(a,b):
    c = [a_i - b_i for a_i, b_i in zip(a, b)]
    return sum(i*i*i*i for i in c)
vector_difference([0,1], [0,2])

1

In [5]:
def vector_add(a,b):
    return [a_i + b_i for a_i, b_i in zip(a, b)]
print(vector_add([0,1], [0,2]))

[0, 3]


In [6]:
#One word input
def get_cat(word):
    vector_diff_list = []
    word_vector = vectors[word]
    for i in cat_vector_list:
        #Get sum of squared differences
        vector_diff_list.append(vector_difference(word_vector, i))
    #Get index of cat_vector with least difference
    min_index = vector_diff_list.index(min(vector_diff_list))
    return cat_list[min_index]

In [7]:
#Multiple word input
def get_cat_phrase(phrase):
    word_list = phrase.split()
    vector_diff_list = []
    number_of_valid_words = 0
    for i in range(len(word_list)):
        word = word_list[i]
        try:
            word_vector = vectors[word]
            number_of_valid_words += 1
        except:
            continue
        
        for j in range(len(cat_vector_list)):
            cat_vector = cat_vector_list[j]
            
            #Get sum of differences
            if number_of_valid_words == 1:
                vector_diff_list.append(vector_difference(word_vector, cat_vector))
            else:
                vector_diff_list[j] += vector_difference(word_vector, cat_vector)
    #Get index of cat_vector with least difference
    min_index = vector_diff_list.index(min(vector_diff_list))
    return cat_list[min_index]

In [33]:
get_cat_phrase("toilet")

'cleanliness'

In [57]:
def get_word_difference(word1, word2):
    return scipy.spatial.distance.cosine(vectors[word1],vectors[word2])

In [141]:
import math
def get_word_similarity(word1, word2):
    diff = get_word_difference(word1, word2)
    return math.exp(-2*diff)

In [142]:
get_word_difference("student", "boris")

0.9801906608045101

In [176]:
get_word_similarity("student", "student")

1.0

In [70]:
type(df_pro['Nouns'][0])

str

In [126]:
sum([1,2,3].divide(6))

AttributeError: 'list' object has no attribute 'divide'

In [232]:
sentence_weight_list = []
for string_list in df_pro['Nouns']:    
    exec("loop_list = " + string_list)
    weight_list = []
    for i in range(len(cat_list)):
        weight_list.append(0)
    for word in loop_list:
        for i in range(len(cat_list)):
            try:
                #print (word)
                similarity_score = get_word_similarity(word, cat_list[i])
                if similarity_score > 0.25:
                    weight_list[i] += similarity_score
            except:
                continue;
        
    #total_weight = sum(weight_list)
    #if total_weight != 0:
    #    weight_list = [weight / total_weight for weight in weight_list]
    sentence_weight_list.append(weight_list)
sentence_weight_list

[[0.5027220146187072, 0, 0.26794897042791166, 0, 0.2696445559596108],
 [1.0, 0, 0, 0, 0],
 [0, 0.8324387540330229, 0.2796058778200253, 0, 0],
 [0, 0.2522576142741331, 0.28613042315504916, 0, 0],
 [0.5140339262401683, 0, 0, 0, 0],
 [1.2961738997816923, 0, 0, 0, 0],
 [0.25421342641102884, 0, 0, 0, 0],
 [1.0, 0, 0, 0, 0],
 [0.2892717069064972, 0, 0.25843665821612355, 0, 0.3189868992847883],
 [0.25217264666663874, 0, 0.30414143566677626, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0.25376079120883127, 0, 0, 0, 0.28390724140821094],
 [0.5309025582027374, 0, 0.6364201488810952, 0, 0.2556428678438321],
 [0.25707303796780484, 0, 0.3028610395488392, 0, 0],
 [0.28210454829698395, 0, 0, 0, 0],
 [0.26049706022822244, 0, 0, 0, 0],
 [0, 0, 0.2555957730625951, 0, 0.263515015831121],
 [0, 0.496338724228668, 0, 0, 0],
 [0.5297798985459325, 0, 0.6349950187067266, 0, 0],
 [1.0, 0, 0, 0, 0],
 [0.5081057999443703, 0, 0.25996542120817223, 0, 0],
 [0, 0.2979909306838254, 0.2654795481935543

In [233]:
df_new = pd.DataFrame(columns=cat_list, data=sentence_weight_list)
df_new

,space,personnel,social,cleanliness,distance
0,0.502722,0.000000,0.267949,0.0,0.269645
1,1.000000,0.000000,0.000000,0.0,0.000000
2,0.000000,0.832439,0.279606,0.0,0.000000
3,0.000000,0.252258,0.286130,0.0,0.000000
4,0.514034,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...
154,0.000000,0.293641,0.267514,0.0,0.000000
155,0.296174,0.496339,0.000000,0.0,0.000000
156,0.000000,0.000000,0.000000,0.0,0.000000
157,0.622179,0.000000,0.000000,0.0,0.000000


In [234]:
test_i = 0

test_i += 1
print(df_pro['Review'].loc[test_i])
print(df_new.loc[test_i])